In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, Volume, AnimationParams, Vector2, Vector3, Quaternion

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()

In [ ]:
generate_snapshot = False
generate_movie = False

status = be.reset_scene()
status = be.set_general_settings(logging_level=3)

''' Simulation identifier '''
simulation_guid = 3

''' Ratio to apply to concentrations for visualization purpose '''
concentration_visualization_ratio = 1.0

representation = be.REPRESENTATION_ATOMS_AND_STICKS
atom_radius_multiplier = 1.0 

scene_size = Vector3(250.0, 500.0, 250.0)
membrane_size = Vector3(scene_size.x, scene_size.y / 50.0, scene_size.z)

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
off_folder = resource_folder + 'off/'
off_folder = '/home/favreau/git/BioExplorer/bioexplorer/pythonsdk/tests/test_files/off'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
ion_channels_folder = pdb_folder + 'ion_channels/'
metabolites_folder = pdb_folder + 'metabolites/'
transporters_folder = pdb_folder + 'transporters/'

be.set_general_settings(
    off_folder=off_folder,
    model_visibility_on_creation=False)

''' Neuron trans-membrane proteins '''
pdb_glut3 = transporters_folder + '4zwc.pdb'
pdb_mct2_lac = transporters_folder + '7bp3.pdb'

''' Astrocyte trans-membrane proteins '''
pdb_glut1 = transporters_folder + '4pyp.pdb'
pdb_mct1_lac = transporters_folder + '6lz0.pdb'

''' Trans-membrane proteins '''
pdb_nka = transporters_folder + '4hqj.pdb'

''' Lipids '''
pdb_lipids = [
    lipids_folder + 'lipid_430.pdb',
    lipids_folder + 'lipid_426.pdb',
    lipids_folder + 'lipid_424.pdb',
    lipids_folder + 'lipid_410.pdb'
]
lipid_density = 3.0

## Metabolites

In [ ]:
import psycopg2
import pandas as pd
import os

db_host = os.environ['DB_HOST']
db_name = os.environ['DB_NAME']
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_schema = os.environ['DB_SCHEMA']

db_connection_string = 'host=' + db_host + ' port=5432 dbname=' + db_name + \
              ' user=' + db_user + ' password=' + db_password
print(db_connection_string)

db_connection=psycopg2.connect(db_connection_string)
print('Connected to ' + db_name + '.' + db_schema)

In [ ]:
def get_simulations():
    simulations = dict()
    sql_command = "SELECT guid, description FROM %s.simulations ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        simulations[data['guid'][i]] = data['description'][i]
    return simulations

def get_variables():
    variables = dict()
    sql_command = "SELECT guid, pdb, description FROM %s.variable WHERE pdb IS NOT NULL ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        pdb_guid = data['pdb'][i]
        if pdb_guid:
            variables[data['guid'][i]] = [pdb_guid, data['description'][i]]
    return variables

def get_metabolites():
    metabolites = dict()
    sql_command = "SELECT guid, pdb, description FROM %s.variable WHERE type=0 AND pdb IS NOT NULL ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        pdb_guid = data['pdb'][i]
        if pdb_guid:
            metabolites[data['guid'][i]] = [pdb_guid, data['description'][i]]
    return metabolites

def get_locations():
    locations = dict()
    sql_command = "SELECT guid, description FROM %s.location ORDER BY guid" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        locations[data['guid'][i]] = data['description'][i]
    return locations

def get_concentration(variable_guid, simulation_guid, frame, location_guid):
    sql_command = 'SELECT v.guid AS guid, c.concentration AS concentration FROM %s.variable as v, %s.concentration AS c WHERE c.variable_guid=%d AND v.guid=c.variable_guid AND c.timestamp=%d AND c.simulation_guid=%d AND v.location_guid=%d ORDER BY v.guid' % (db_schema, db_schema, variable_guid, frame, simulation_guid, location_guid)
    data = pd.read_sql(sql_command, db_connection)
    if(len(data)>0):
        return(float(data['concentration']))
    return 0.0    

In [ ]:
''' Dimensions are in nanometers '''

import math

region_astrocyte_mitochondrion = 0
region_astrocyte = 1
region_extracellular_space = 2
region_neuron = 3
region_neuron_mitochondrion = 4

region_mapping = dict()
region_mapping[0] = region_astrocyte_mitochondrion
region_mapping[1] = region_astrocyte
region_mapping[2] = region_extracellular_space
region_mapping[3] = region_neuron
region_mapping[4] = region_neuron_mitochondrion

location_areas = dict()
location_areas[region_astrocyte_mitochondrion] = [0.0575 * 0.25, Vector2(100, scene_size.x / 2.0)] # Mitochondria of the astrocyte
location_areas[region_astrocyte] = [0.25, Vector2(20, 100)] # Cytosol of the astrocyte
location_areas[region_extracellular_space] = [0.20, Vector2(-20, 20)] # Extracellular space
location_areas[region_neuron] = [0.45, Vector2(-10, -100)] # Cytosol of the neuron
location_areas[region_neuron_mitochondrion] = [0.0575 * 0.45, Vector2(-100, -scene_size.x / 2.0)] # Mitochondria of the neuron


# Currently not used:
# -------------------
# region_capilarities = 5
# region_synaptic = 6
# region_vasculature = 7
# -------------------
# location_areas[region_capilarities] = [0.0055, Vector2(0, 0)] # Capillaries
# location_areas[region_synaptic] = [0.0, Vector2(0, 0)] # Synaptic part of the extracellular space
# location_areas[region_vasculature] = [0.0, Vector2(0, 0)] # Vasculature
# -------------------


In [ ]:
fullNGVUnitVolumeInLiters = 2e-11 / 0.45
nanometersCubicToLiters = 1e-24
avogadro = 6.02e23

fullSceneVolumeInLiters = scene_size.x * scene_size.y * scene_size.z * nanometersCubicToLiters
scene_ratio = fullSceneVolumeInLiters / fullNGVUnitVolumeInLiters

In [ ]:
def get_nb_molecules(concentration, location_guid):
    return int(concentration_visualization_ratio * scene_ratio * avogadro * (1e-3 * concentration) * fullNGVUnitVolumeInLiters * location_areas[location_guid][0])

def get_nb_proteins(concentration, location_guid):
    return int(math.exp(concentration) * 1e-6 * avogadro * fullNGVUnitVolumeInLiters * scene_ratio * location_areas[location_guid][0])

In [ ]:
indices = list()
indices.append(-scene_size.y / 2.0)
total = 0.0
for location_area in location_areas:
    total = total + location_areas[location_area][0] * scene_size.y
    indices.append(-scene_size.y / 2.0 + total)
indices.append(scene_size.y / 2.0)
# print(indices)

i = 0
for region in region_mapping:
    location_areas[region_mapping[region]][1] = Vector2(indices[i], indices[i+1])
    i += 1


In [ ]:
def add_metabolites(frame=0):
    locations = get_locations()
    variables = get_metabolites()

    random_seed = 1
    for location in locations:
        if location not in region_mapping.values():
            continue

        for variable in variables:
            pdb_guid = variables[variable][0]
            if not pdb_guid:
                continue
            variable_guid = int(variable)
            variable_description = variables[variable][1]
            location_guid = int(location)
            file_name = metabolites_folder + pdb_guid + '.pdb'
            concentration = get_concentration(variable_guid, simulation_guid, frame, location_guid)
            nb_molecules = get_nb_molecules(concentration, location_guid)
            if nb_molecules > 0:
                location_name = locations[location_guid]
                print('- [%s] [%d] %s: %s.pdb: %d' % (location_name, variable_guid, variable_description, pdb_guid, nb_molecules))
                try:
                    location_area = location_areas[location_guid][1]
                    area_size = Vector3(
                        scene_size.x,
                        0.95 * (location_area.y - location_area.x),
                        scene_size.z)
                    area_position = Vector3(0.0, (location_area.y + location_area.x) / 2.0 , 0.0)

                    name = location_name + '_' + variable_description
                    metabolite = Protein(
                        name=name, source=file_name,
                        load_bonds=True, load_hydrogen=True,
                        load_non_polymer_chemicals=True, 
                        occurences=nb_molecules,
                        animation_params=AnimationParams(
                            random_seed,
                            random_seed + frame + 1, 0.25, 
                            random_seed + frame + 2, 1.0))

                    volume = Volume(
                        name=name,
                        shape=be.ASSEMBLY_SHAPE_CUBE,
                        shape_params=area_size,
                        protein=metabolite
                    )
                    status = be.add_volume(
                        volume=volume, representation=representation, position=area_position)
                    random_seed += 3
                except Exception as e:
                    print(e)

## Neuron

In [ ]:
def add_neuron(frame=0):
    name = 'Neuron'
    transmembrane_proteins = list()

    # Transporter GLUT3 (https://opm.phar.umich.edu/proteins/442)
    nb_transporter_glut3 = get_nb_proteins(-0.250079574048427, region_neuron)
    if nb_transporter_glut3 > 0:
        print('- Transpoter GLUT3: %d' % nb_transporter_glut3)
        transmembrane_proteins.append(Protein(
            name=name + '_GLUT3',
            chain_ids=[1],
            source=pdb_glut3,
            occurences=30, #nb_transporter_glut3,
            rotation=Quaternion(0.707, -0.693, -0.139, 0.0),
            load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
            animation_params=AnimationParams(1, frame + 1, 0.025, frame + 2, 0.005),
            transmembrane_params=Vector2(0.0, 2.0)
        ))

    # Transporter MCT2 (https://opm.phar.umich.edu/proteins/5233)
    nb_transporter_mct2 = get_nb_proteins(-2.00832096285463, region_neuron)
    if nb_transporter_mct2 > 0:
        print('- Transporter MCT2: %d' % nb_transporter_mct2)
        transmembrane_proteins.append(Protein(
            name=name + '_MCT2',
            position=Vector3(0.0, 1.0, 0.0),
            source=pdb_mct2_lac,
            occurences=nb_transporter_mct2,
            load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
            animation_params=AnimationParams(2, frame + 3, 0.025, frame + 4, 0.005),
            transmembrane_params=Vector2(0.0, 2.0)
        ))

    # Transporter NKA
    nb_transporter_nka = get_nb_proteins(0.325597445694269, region_neuron)
    if nb_transporter_nka > 0:
        print('- Transporter NKA: %d' % nb_transporter_nka)
        transmembrane_proteins.append(Protein(
            name=name + '_NKA',
            source=pdb_nka,
            occurences=nb_transporter_nka,
            load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
            animation_params=AnimationParams(3, frame + 5, 0.025, frame + 6, 0.005),
            transmembrane_params=Vector2(0.0, 2.0)
        ))
    

    # Membrane definition
    membrane = Membrane(
        lipid_sources=pdb_lipids,
        lipid_density=lipid_density,
        load_non_polymer_chemicals=True, load_bonds=True,
        animation_params=AnimationParams(0, frame + 7, 0.025, frame + 8, 0.005)
    )

    # Cell definition
    neuron = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=membrane_size,
        membrane=membrane,
        proteins=transmembrane_proteins)

    # Add cell to scene
    status = be.add_cell(
        cell=neuron, representation=representation,
        atom_radius_multiplier=atom_radius_multiplier,
        position=Vector3(0, location_areas[region_neuron][1].x, 0))

    return status

In [ ]:
def add_neuron_mitochondrion(frame):
    name = 'NeuronMitochondrion'

    # Transporter
    transporter = Protein(
        name=name + '_GLUT3',
        source=pdb_glut3,
        occurences=0,
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
        animation_params=AnimationParams(4, frame + 9, 0.025, frame + 10, 0.005),
        transmembrane_params=Vector2(0.0, 2.0)
    )

    # Membrane definition
    membrane = Membrane(
        lipid_sources=pdb_lipids,
        lipid_density=lipid_density,
        load_non_polymer_chemicals=True, load_bonds=True,
        animation_params=AnimationParams(0, frame + 11, 0.025, frame + 12, 0.005)
    )

    # Cell definition
    neuron_mitochodrion = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=membrane_size,
        membrane=membrane,
        proteins=[transporter])

    # Add cell to scene
    status = be.add_cell(
        cell=neuron_mitochodrion, representation=representation,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        position=Vector3(0, location_areas[region_neuron_mitochondrion][1].x, 0))

    return status

## Astrocyte

In [ ]:
def add_astrocyte(frame = 0):
    name = 'Astrocyte'
    transmembrane_proteins = list()

    # GLUT1 (https://opm.phar.umich.edu/proteins/2454)
    nb_transporter_glut1 = get_nb_proteins(-0.672647584446565, region_neuron)
    if nb_transporter_glut1 > 0:
        print('- Transporter GLUT1: %d' % nb_transporter_glut1)
        transmembrane_proteins.append(Protein(
            name=name + '_GLUT1',
            source=pdb_glut1,
            occurences=nb_transporter_glut1,
            rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
            load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
            animation_params=AnimationParams(11, frame + 13, 0.025, frame + 14, 0.005),
            transmembrane_params=Vector2(0.0, 2.0)
        ))

    # Transporter MCT1 (https://opm.phar.umich.edu/proteins/6402)
    nb_transporter_mct1 = get_nb_proteins(-0.86948422680206, region_neuron)
    if nb_transporter_mct1 > 0:
        print('- Transporter MCT1:  %d' % nb_transporter_mct1)
        transmembrane_proteins.append(Protein(
            name=name + '_MCT1',
            position=Vector3(0.0, 1.0, 0.0),
            source=pdb_mct1_lac,
            occurences=nb_transporter_mct1,
            load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
            animation_params=AnimationParams(12, frame + 15, 0.025, frame + 16, 0.005),
            transmembrane_params=Vector2(0.0, 2.0)
        ))

    # Membrane definition
    membrane = Membrane(
        lipid_sources=pdb_lipids,
        lipid_density=lipid_density,
        load_non_polymer_chemicals=True, load_bonds=True,
        animation_params=AnimationParams(0, frame + 17, 0.025, frame + 18, 0.005))

    # Cell definition
    astrocyte = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=membrane_size,
        membrane=membrane,
        proteins=transmembrane_proteins)

    # Add cell to scene
    status = be.add_cell(
        cell=astrocyte, representation=representation,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        position=Vector3(0, location_areas[region_astrocyte][1].y, 0))

    return status

In [ ]:
def add_astrocyte_mitochondrion(frame=0):
    name = 'AstrocyteMitochondrion'

    # Transporter
    transporter = Protein(
        name=name + '_GLUT1',
        source=pdb_glut1,
        occurences=0,
        rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
        load_non_polymer_chemicals=True, load_hydrogen=True, load_bonds=True,
        animation_params=AnimationParams(15, frame + 19, 0.025, frame + 20, 0.005),
        transmembrane_params=Vector2(0.0, 2.0)
    )

    # Membrane definition
    membrane = Membrane(
        lipid_sources=pdb_lipids,
        lipid_density=lipid_density,
        load_non_polymer_chemicals=True, load_bonds=True,
        animation_params=AnimationParams(0, frame + 21, 0.025, frame + 22, 0.005)
    )

    # Cell definition
    astrocyte_mitochodrion = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=membrane_size,
        membrane=membrane,
        proteins=[transporter])

    # Add cell to scene
    status = be.add_cell(
        cell=astrocyte_mitochodrion, representation=representation,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        position=Vector3(0, location_areas[region_astrocyte_mitochondrion][1].y, 0))

    return status

## Materials

In [ ]:
def set_materials_to_transmembrane_proteins():
    name = 'Neuron'
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name+'_GLUT3',
        color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
        palette_name='Reds_r', palette_size=2)

    status = be.set_protein_color_scheme(
        assembly_name=name, name=name+'_MCT2',
        color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
        palette_name='Greens_r', palette_size=2)

    status = be.set_protein_color_scheme(
        assembly_name=name, name=name+'_NKA',
        color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
        palette_name='OrRd_r', palette_size=2)

    name = 'Astrocyte'
    status = be.set_protein_color_scheme(
        assembly_name=name, name=name+'_GLUT1',
        color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
        palette_name='Reds_r', palette_size=2)

    status = be.set_protein_color_scheme(
        assembly_name=name, name=name+'_MCT1',
        color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
        palette_name='Greens_r', palette_size=2)

In [ ]:
def set_color_scheme(
        shading_mode, user_parameter=1.0, specular_exponent=5.0, glossiness=1.0):
    """
    Apply a default color scheme to all components in the scene

    :shading_mode: Shading mode (None, basic, diffuse, electron, etc)
    :user_parameter: User parameter specific to each shading mode
    :specular_exponent: Specular exponent for diffuse shading modes
    :glossiness: Glossiness
    """

    import seaborn as sns
    model_ids = be.get_model_ids()
    global_palette = sns.color_palette('rainbow', len(model_ids["ids"]))

    index = 0
    for model_id in model_ids["ids"]:
        model_name = be.get_model_name(model_id)['name']
        material_ids = be.get_material_ids(model_id)["ids"]
        nb_materials = len(material_ids)

        if model_name.find('Neuron') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Blues", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        elif model_name.find('Astrocyte') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Wistia", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        elif model_name.find('Mitochondrion') != -1:
            palette = sns.color_palette("Purples", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        else:
            colors = list()
            shading_modes = list()
            user_parameters = list()
            glossinesses = list()
            specular_exponents = list()

            for m in material_ids:
                colors.append(global_palette[index])
                shading_modes.append(shading_mode)
                user_parameters.append(user_parameter)
                glossinesses.append(glossiness)
                specular_exponents.append(specular_exponent)

            be.set_materials(
                model_ids=[model_id],
                material_ids=material_ids,
                diffuse_colors=colors,
                specular_colors=colors,
                shading_modes=shading_modes,
                user_parameters=user_parameters,
                glossinesses=glossinesses,
                specular_exponents=specular_exponents
            )
        index += 1

In [ ]:
def add_aabb():
    return be.add_bounding_box(
        name='AABB',
        bottom_left_corner=Vector3(-scene_size.x / 2.0, -scene_size.y / 2.0, -scene_size.z / 2.0),
        top_right_corner=Vector3(scene_size.x / 2.0, scene_size.y / 2.0, scene_size.z / 2.0),
        radius=0.25)

In [ ]:
def setup_frame(frame, metabolites=True, neuron=True, astrocyte=True):
    be.reset_scene()
    be.set_general_settings(model_visibility_on_creation=False)
    print('Scene bounding box...')
    add_aabb()
    if metabolites:
        print('Loading metabolites...')
        add_metabolites(frame % 450)
    if astrocyte:
        print('Loading astrocyte mitochondrion membrane...')
        add_astrocyte_mitochondrion(frame)
        print('Loading astrocyte membrane...')
        add_astrocyte(frame)
    if neuron:
        print('Loading neuron membrane...')
        add_neuron(frame)
        print('Loading neuron mitochondrion membrane...')
        add_neuron_mitochondrion(frame)
    print('Applying materials...')
    set_color_scheme(shading_mode=be.SHADING_MODE_PERLIN, user_parameter=0.001, specular_exponent=50.0)
    set_materials_to_transmembrane_proteins()
    print('Building geometry...')
    be.set_models_visibility(True)
    print('Done')

In [ ]:
be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

params = core.BioExplorerRendererParams()
params.shadows = 1.0
params.soft_shadows = 0.1
params.use_hardware_randomizer = True
params.fog_start = 1000.0
params.fog_thickness = 500.0
params.gi_distance = 50.0
params.gi_weight = 0.2
params.gi_samples = 1
params = core.set_renderer_params(params)
params = core.set_renderer(head_light=True)

## Snapshots

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
if generate_snapshot:
    core.set_camera(current='orthographic')
    params = core.OrthographicCameraParams()
    params.height = 100.0
    core.set_camera_params(params)

    core.set_camera(
        orientation = [-0.707, -0.707, 0.0, 0.0],
        position = [0.7841586723976661, 3.6684584451627895, -360.03218098964624],
        target = [0.7841586723976661, 3.6684584451626847, -25.817442464876535]
    )

    mm.create_snapshot(
        renderer='bio_explorer',
        base_name = 'Metabolism_Molecular_Level_4k_v10',
        path='/gpfs/bbp.cscs.ch/project/proj129/images/metabolism/4K',
        size=[2160, 3840], samples_per_pixel=64
)

## Simulation movie

In [ ]:
nb_frames = 60
aperture = 0.01
focus = 2.938
key_frames = [
    {
        'apertureRadius': aperture,
        'direction': [-0.0, 0.0, -1.0],
        'focusDistance': focus,
        'origin': [-0.12309164325388476, -68.17223360869282, 528.224633426888],
        'up': [-1.0, -6.661338147750939e-16, 0.0]
    },
    {
        'apertureRadius': aperture,
        'direction': [-0.0, 0.0, -1.0],
        'focusDistance': focus,
        'origin': [-0.12309164325388476, -68.17223360869282, 140.11376864318507],
        'up': [-1.0, -6.661338147750939e-16, 0.0]
    },
    {
        'apertureRadius': aperture,
        'direction': [0.0, 1.0, -2.220446049250313e-16],
        'focusDistance': focus,
        'origin': [-28.5, -67.86393737792969, -2.0],
        'up': [-0.0, 2.220446049250313e-16, 1.0]
    },
    {
        'apertureRadius': aperture,
        'direction': [-0.0, 1.0, -4.440892098500626e-16],
        'focusDistance': focus,
        'origin': [-29.22458300330654, -18.82142928630076, -2.386302379782111],
        'up': [0.0, 4.440892098500626e-16, 1.0]
    },
    {
        'apertureRadius': aperture,
        'direction': [-0.0, 1.0, -2.220446049250313e-16],
        'focusDistance': focus,
        'origin': [-29.22458300330654, -0.82142928630076, -2.386302379782111],
        'up': [0.0, 2.220446049250313e-16, 1.0]
    },
    {
        'apertureRadius': aperture,
        'direction': [0.9999970255814865, 1.5866404717910239e-21, -0.0024390219719829798],
        'focusDistance': focus,
        'origin': [-117.00812364660472, -14.7012431105328, -16.27792336717559],
        'up': [0.0, 1.0, 6.505232384196665e-19]
    }
]
mm.build_camera_path(key_frames, nb_frames, nb_frames)

### Get the model that contains the glucose molecules

In [ ]:
glucose_model_name = 'Extracellular space_Glucose'
glucose_model_id = -1
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    model_name = be.get_model_name(model_id)['name']
    if model_name==glucose_model_name:
        glucose_model_id = model_id

print(glucose_model_id)

In [ ]:
import time
import time
from IPython.display import clear_output

draft = False
spp = 64
k = 2
output_folder = '/tmp'

if generate_movie:
    core.set_application_parameters(image_stream_fps=0)
    core.set_renderer(current='bio_explorer')
    
    for frame in range(mm.get_nb_frames()):
        clear_output()
        print('Frame %d' % frame)
        
        setup_frame(frame)
        mm.set_current_frame(frame)

        ''' Set glucose molecule position '''
        p = core.get_camera()['position']
        t = core.get_camera()['target']
        
        pos = [0,0,0]
        for i in range(3):
            pos[i] = p[i] + (t[i] - p[i]) * 3.0
        
        target = Vector3(pos[0], pos[1], pos[2])
        glucose_model_name = 'Extracellular space_Glucose'
        be.set_protein_instance_transformation(
            assembly_name=glucose_model_name,
            name=glucose_model_name,
            instance_index=0,
            position=target, rotation=Quaternion.random())    

        ''' Set depth of field '''
        key_frame = mm.get_key_frame(frame)
        params = core.BioExplorerPerspectiveCameraParams()
        params.aperture_radius = key_frame[3]
        params.focus_distance = key_frame[4]
        core.set_camera_params(params)
        core.set_camera(current='bio_explorer_perspective')

        '''Generate image'''
        if draft:
            time.sleep(0.1)
        else:
            mm.create_snapshot(
                renderer='bio_explorer',
                size=[k * 960, k * 540], samples_per_pixel=spp,
                path=output_folder, base_name='%05d' % frame)
    core.set_application_parameters(image_stream_fps=20)
else:
    setup_frame(frame=0, metabolites=True, astrocyte=True, neuron=True)       